# COHORTS SSIM

In [1]:
import torch
from datasets import BrainImages, SingleMod


testLoader = "/home/tbarba/projects/MultiModalBrainSurvival/outputs/norm/AE_sel2/exports/testLoader.pth"
testLoader = torch.load(testLoader)

/share/pi/ogevaert/thomas/miniconda/envs/multimodal/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import src.autoencoder.models.classicalAE as cae
import src.autoencoder.models.VAEs as vae

import torch
import torch.nn as nn
import nibabel as nib
import torchio as tio
import numpy as np
import matplotlib.pyplot as plt
from monai.losses.ssim_loss import SSIMLoss


ModuleNotFoundError: No module named 'src'

In [ ]:
model = "AE_sel2"
type_ae = model.split("_")[0].lower()

if type_ae == "vae3d":
    net = vae.VAE3D([182, 218, 160], 1, latent_dim=2048)
elif type_ae == "uvae":
    net = vae.U_VAE(in_channels=1,  init_features=4,
                    num_blocks=6, input_dim=[182, 218, 160],
                    attention=None, hidden_size=2048)
else:
    net = cae.BaseAE(in_channels=1,
                     init_features=4, num_blocks=6, type_ae=type_ae,
                     dropout=0.1, attention=None)


net = nn.DataParallel(net)
bckp = torch.load(f"/home/tbarba/projects/MultiModalBrainSurvival/outputs/norm/{model}/exports/best_model.pt")
net.load_state_dict(bckp)

================> [4, 8, 16, 32, 64, 128] 256


<All keys matched successfully>

In [ ]:
def compute_ssim(orig_nifti, output):
    ssim_func = SSIMLoss(spatial_dims=3)

    imgs = nib.load(orig_nifti)
    rescale = tio.RescaleIntensity(out_min_max=(0, 1))
    imgs = rescale(imgs).get_fdata()
    imgs = np.array(imgs, dtype=np.float16)
    imgs = np.expand_dims(imgs, axis=0)  # adding channel dim
    imgs = imgs.transpose((0, 3, 2, 1))  # reorder channels
    imgs = np.expand_dims(imgs, axis=0)  # adding batch dim
    orig = torch.Tensor(imgs)

    ssim_loss = ssim_func(orig, output, data_range=orig.max()).item()
    ssim = 1 - ssim_loss
    
    return ssim


net.eval()

image = "/home/tbarba/projects/MultiModalBrainSurvival/data/MR/TCGA/processed/TCGA-HT-8114/normT2_crop.nii.gz"
orig = nib.load(image).get_fdata()
output, embedding = net.module.process_nifti(image)
reconstructed = output.squeeze().detach().numpy().transpose((2,1,0))
images = {"orig":orig, "recontr":reconstructed}

ssim = compute_ssim(image, output)



In [4]:
from datasets import SingleMod

a = torch.load("/home/tbarba/projects/MultiModalBrainSurvival/data/outputs/norm/AE_sel2/exports/testLoader.pth")

In [10]:
testLoader.num_workers = 0

In [20]:
from tqdm import tqdm

test_cohort = []
for i, a in tqdm(enumerate(testLoader)):
    test_cohort.extend(list(set([f.split("__")[0] for f in a[1]])))

55it [06:28,  7.06s/it]


In [24]:
import pandas as pd
pd.DataFrame(test_cohort).to_csv("test_loader.csv", index=False)

In [25]:
len(test_cohort)

760